<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Иследование-качества-модели-без-преобразования" data-toc-modified-id="Иследование-качества-модели-без-преобразования-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Иследование качества модели без преобразования</a></span></li><li><span><a href="#Иследование-качества-модели-c-преобразованием" data-toc-modified-id="Иследование-качества-модели-c-преобразованием-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Иследование качества модели c преобразованием</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# План работы
1) Загрузите и изучите данные.

2) Ответьте на вопрос и обоснуйте решение.
- Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
        -a. Изменится. Приведите примеры матриц.
        -b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
        
3) Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

4) Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
insurance_df = pd.read_csv('/datasets/insurance.csv')
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
# Проверим себя и напишем функцию для определения уникальных значений
def show_rows(data_frame):
    for column in data_frame.columns:
        print('Уникальные значения столбца', column)
        print(data_frame[column].unique())
    print('Количество пропусков в столбце')    
    print(data_frame.isna().mean())

In [4]:
show_rows(insurance_df)

Уникальные значения столбца Пол
[1 0]
Уникальные значения столбца Возраст
[41. 46. 29. 21. 28. 43. 39. 25. 36. 32. 38. 23. 40. 34. 26. 42. 27. 33.
 47. 30. 19. 31. 22. 20. 24. 18. 37. 48. 45. 44. 52. 49. 35. 56. 65. 55.
 57. 54. 50. 53. 51. 58. 59. 60. 61. 62.]
Уникальные значения столбца Зарплата
[49600. 38000. 21000. 41700. 26100. 41000. 39700. 38600. 49700. 51700.
 36600. 29300. 39500. 55000. 43700. 23300. 48900. 33200. 36900. 43500.
 36100. 26600. 48700. 40400. 38400. 34600. 34800. 36800. 42200. 46300.
 30300. 51000. 28100. 64800. 30400. 45300. 38300. 49500. 19400. 40200.
 31700. 69200. 33100. 31600. 34500. 38700. 39600. 42400. 34900. 30500.
 24200. 49900. 14300. 47000. 44800. 43800. 42700. 35400. 57200. 29600.
 37400. 48100. 33700. 61800. 39400. 15600. 52600. 37600. 52500. 32700.
 51600. 60900. 41800. 47400. 26500. 45900. 35700. 34300. 26700. 25700.
 33300. 31100. 31500. 42100. 37300. 42500. 27300. 46800. 33500. 44300.
 41600. 53900. 40100. 44600. 45000. 32000. 38200. 33000. 38500

In [5]:
insurance_df.head(3)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


**Вывод** Видим что в столбцах `Возраст` и `Зарплата` тип данных float, хотя значения целые, так что поменяем их для уменьшения памяти и быстродеятсвия программы.

In [6]:
insurance_df['Возраст'] = pd.to_numeric(insurance_df['Возраст'], downcast='integer')
insurance_df['Зарплата'] = pd.to_numeric(insurance_df['Зарплата'], downcast='integer')

In [7]:
#insurance_df.head(3)

In [8]:
insurance_df[['Зарплата', 'Возраст']] = insurance_df[['Зарплата', 'Возраст']].astype('int')

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Нам необхлдимо доказать что $$a = a_p$$
следовательно должно получится $$Xw = PXw_p$$

Формула обучения:


$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
$$
w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y = (P^T (X^T X) P)^{-1} P^T X^T y = (X^TX)^{-1}(P^TP)^{-1}P^T X^T y = (X^TX)^{-1}P^{-1} (P^T)^{-1}P^T X^T y =(X^TX)^{-1}P^{-1} X^T y => w' = P^-1 w
$$

**Вывод** Значение предсказания не меняется если просходит перемножение признаков на обратимую матрицу

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится

**Обоснование:** Приведено выше

## Алгоритм преобразования

**Алгоритм**

Для защиты данных мы будем умножать матрицу признаков на обратимую матрицу, которая будет генерироваться случайно. Своего рода СМС-код при двухэтапной идентификации.

**Обоснование**
![](https://pictures.s3.yandex.net/resources/b_1578838605.jpg)</div>

## Проверка алгоритма

### Иследование качества модели без преобразования

In [9]:
# Выделим признаки и целевой признак
features = insurance_df.drop('Страховые выплаты',axis=1)
target = insurance_df['Страховые выплаты']

In [10]:
# Создаем выборки обуч и тест
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=1515)

In [11]:
# Модель с исходными признаками без преобразования
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_origin_data = r2_score(target_test, model.predict(features_test))
print("R2 =", R2_LR_origin_data)

R2 = 0.4543201571260044


In [12]:
# Модель с отмасштабированными признаками без преобразования
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_origin_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_origin_data_scaled)

R2 = 0.4543201571260026


Видим что качество модели не меняется (в самом конце есть небольшое различие, откуда оно?. Почитал в Слаке, что отличие после 12 знака "незначимо"

### Иследование качества модели c преобразованием

In [13]:
# Создадим рандомную матрицу-ключ
n = features.shape[1]
crypto_matrix = np.random.randint(1,10, (n,n))
#Признаки защищенные
crypto_features = features
crypted_features = crypto_features @ crypto_matrix
np.linalg.inv(crypto_matrix)

array([[ -7.71428571,  11.14285714,  -5.71428571,   8.14285714],
       [ -9.        ,  13.        ,  -7.        ,  10.        ],
       [  5.28571429,  -7.85714286,   4.28571429,  -5.85714286],
       [ 12.        , -17.        ,   9.        , -13.        ]])

In [14]:
display(features.head())
display(crypted_features.head())
crypto_matrix

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


,0,1,2,3
0,248168,446622,446533,297816
1,190185,342238,342144,228234
2,105116,189145,189087,126145
3,208586,375421,375375,250313
4,130615,235049,234988,156747


array([[3, 9, 4, 7],
       [4, 5, 3, 5],
       [5, 9, 9, 6],
       [1, 8, 6, 4]])

Видим что вместо оригинальных данных теперь находятся неопнятные числа, далее также разобьем данные на трен и тест выборку и проверим качество

In [15]:
features_train, features_test, target_train, target_test = train_test_split(
    crypted_features, target, test_size=0.25, random_state=1515)

In [16]:
# Модель с исходными признаками без преобразования
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_cipher_data = r2_score(target_test, model.predict(features_test))
print("R2 =", R2_LR_cipher_data)

R2 = 0.45432015712461515


In [17]:
# Модель с отмасштабированными признаками без преобразования
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_cipher_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_cipher_data_scaled)

R2 = 0.45432015712642493


# Вывод

In [18]:
result = pd.DataFrame(data= [R2_LR_origin_data_scaled,
                      R2_LR_origin_data,
                      R2_LR_cipher_data,
                      R2_LR_cipher_data_scaled], 
                      columns=['R2'], 
                      index=['Качество на исходных данных',
                            'Качество на исходных данных c масштабом',
                            'Качество на преобразованных признаках',
                            'ЛКачество на преобразованных признаках с масштабом',])
result

,R2
Качество на исходных данных,0.45432
Качество на исходных данных c масштабом,0.45432
Качество на преобразованных признаках,0.45432
ЛКачество на преобразованных признаках с масштабом,0.45432


По итогу было выполнено:

1) Данные были загружены и проанализированы

2) Был разработан алгоритм преобразования данных и применен на данных.

3)  **Качество линейной регресии не изменилось от использования исxодной матрицы и исходной матрицы, умноженную на обратимую** Это мы смогли доказать по формуле, а также практическим путем (была исследована метрика R2).

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования